In [1]:
import cpwtools
reload(cpwtools)
import qubit
reload(qubit)
inductiveCoupling = __import__("Transmission Lines.inductiveCoupling")
import numpy as np
from scipy.constants import mu_0, epsilon_0, pi, hbar, e, c
from scipy.constants import physical_constants
from scipy.optimize import fsolve, root
from copy import deepcopy

%matplotlib inline

import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

The inductance per unit length is 2.33e-08 H/m


# Charge Noise Mask Design #

General Notes:
 * Looking at the chip left to right, top to bottom, we have Q1-Q4.  
 * Q4 is the charge sensitive qubit.  All others are normal transmons. 
 * Resonators spaced by ~100MHz, lowest to highest

Still to do:  
 * simulate mutual   
 * bandaids - notches in ground plane?
 * figure out where junctions go
 * loops at end of inductive coupling?
 * dielectric liftoff now?


40	40.7	46.3	1.5	39.0	d=15 t=90 m=1.5  
50	52.9	47.0	1.44	30.9	d=15 t=70 m=1  
60	69	46.9	1.47	32.8	d=15 t=55 m=1  
70	90.5	47.0	1.45	35.6	d=15 t=45 m=1  
80	123	47	1.46	41.9	d=15 t=40 m=1  
90	182	47	1.49	54.1	d=15 t=38 m=1  
100	305	46.9	1.42	74.0	d=15 t=35 m=1  
108	500	46.9	1.48	108	d=15 t=35 m=1  

In [2]:
ri   = (40,   50,   60,   70,   80,   90,   100,  108)
ro   = (40.7, 52.9, 69,   90.5, 123,  182,  305,  500)
Cq   = (46.3, 47.0, 46.9, 47.0, 47.0, 47.0, 46.9, 46.9)
#Cq   = (46.3, 49.5, 46.9, 49.5, 47.0, 51.7, 46.9, 54)
Cg   = (1.5,  1.44, 1.47, 1.45, 1.46, 1.49, 1.42, 1.48)
Cgnd = (39.0, 30.9, 32.8, 35.6, 41.9, 54.1, 74.0, 108)
r_xy = (np.nan, 110, np.nan, 175, np.nan, 290, np.nan, 560)

qubits = []
for i in range(len(ri)):
    q = qubit.Qubit('OCSQubit{:03}'.format(ri[i]))
    q.C_g = Cg[i]*1e-15
    q.C_q = Cq[i]*1e-15
    q.C_resToGnd = Cgnd[i]*1e-15
    q.cpw_length = (700-(ro[i]+15+20+1*ri[i]))*1e-6 # ro + d + 2*g + m*ri
    qubits.append(q)

In [3]:
selectedQubits = [deepcopy(qubits[i]) for i in (5, 1, 7, 3)]

In [11]:
# The qubits that are actually to be used
ri = (108, 108, 108, 108)
ro = (500, 500, 500, 500)
Cq = (46.9, 46.9, 46.9, 46.9)
Cg = (1.48, 1.48, 1.48, 1.48)
Cgnd = (108, 108, 108, 108)
r_xy = (560, 560, 560, 560)

selectedQubits = (0, 1, 2, 3)

qubits = []
for i in range(len(ri)):
    q = qubit.Qubit('OCSQubit{:03}'.format(ri[i]))
    q.C_g = Cg[i]*1e-15
    q.C_q = Cq[i]*1e-15
    q.C_resToGnd = Cgnd[i]*1e-15
    q.cpw_length = (700-(ro[i]+15+20+1*ri[i]))*1e-6 # ro + d + 2*g + m*ri
    qubits.append(q)

In [10]:
# calculate the capacitance from the anharm
[1e15*(e**2/2/a)/2e6/pi/hbar for a in (406,410,362,746)]

[47.70992445086579, 47.24446177329636, 53.50892079295997, 25.96545486199934]

## CPW ##
We want to use the same cpw dimensions for resonator and feedline/purcell filter cpw's so the kinetic inductance correction is the same for everything.

In [5]:
cpw = cpwtools.CPW(material='al', w=10., s=7.)
print cpw

Cl = 0.000164468902665 pF/um
Ll = 0.000432649750216 nH/um
vph = 1.1854681054e8 m/s
z0 = 51.2892479688 Ohms


## $\lambda/4$ readout resonators ##
IMPAs from Google will be good in the 4-6GHz range.  We will aim for resonators near 6GHz, but have a spread from 5-6.5GHz on the mask.  They should be spread every 30MHz or so.
We are changing to fixed resonator frequencies (non variable) for this mask.

The frequency is brought down significantly by the capacitance through to ground through the qubit, as well as the self-capacitance of the coupling cap to ground.  These capacitances pull down the transmon frequency more, so we will set Q3 to have no extension, and set the other qubit frequencies around it.

In [12]:
quarter_curve = 2*pi*50/4
coupling_length = 200   # this coupling length varies depending on desired Qc.
tot_length = quarter_curve*(1+2+2+2+2) + 2*850 + (1335-700) + 200 + 350 + coupling_length 

def L4FromQubit(q):
    L4 = cpwtools.QuarterLResonator(cpw, tot_length)
    seriesCap = q.C_g*q.C_q/(q.C_g+q.C_q)
    L4.addCapacitiveCoupling('g', seriesCap, Z0 = 0)
    L4.addCapacitiveCoupling('c_coupler', q.C_resToGnd, Z0 = 0)
    return L4
    
f0 = 6.0e9

print('How much do the capacitors and change in res length of the qubits pull the resonance?')
print('{:>11}   {:>9}  {:>8}  {:>8}'.format('', 'length', 'f_l', 'C_r'))
L4 = L4FromQubit(qubits[1])
l0 = L4.setLengthFromFreq(f0) - qubits[1].cpw_length
for q in qubits:
    L4 = L4FromQubit(q)
    L4.l = l0 + q.cpw_length
    q.C_r = L4.C()
    q.omega_r = L4.wl()
    q.omega_q = 2*pi*(f0-1e9)
    print('{:>11}:  {:>7.2f}um  {:>5.3f}GHz  {:>6.2f}fF'.format(
        q.name, 1e6*l0, L4.fl()/1e9, 1e15*L4.C()))

print
base_length = quarter_curve*(1+2+2+2+2) + (1335-700) + 2*600 + (533-392) + coupling_length  
res_lengths = [base_length + 2*l for l in (728, 684, 641, 599)]
Deltas = (1, 1., 1, 1.)
for i,q in enumerate(selectedQubits):
    L4 = L4FromQubit(q)
    L4.l = 1e-6*res_lengths[i] + q.cpw_length
    q.C_r = L4.C()
    q.omega_r = L4.wl()
    q.omega_q = 2*pi*(L4.fl()-Deltas[i]*1e9)
    print L4.fl()/1e9, q.omega_r/2e9/pi, q.omega_q/2e9/pi

How much do the capacitors and change in res length of the qubits pull the resonance?
                 length       f_l       C_r
OCSQubit040:  4184.88um  5.917GHz  432.64fF
OCSQubit050:  4184.88um  6.000GHz  422.66fF
OCSQubit060:  4184.88um  6.018GHz  422.44fF
OCSQubit070:  4184.88um  6.037GHz  422.64fF
OCSQubit080:  4184.88um  6.044GHz  425.45fF
OCSQubit090:  4184.88um  6.043GHz  432.00fF
OCSQubit100:  4184.88um  6.071GHz  440.90fF
OCSQubit108:  4184.88um  6.096GHz  458.26fF

5.859006199141345 5.859006199141344 4.859006199141345
5.920850720841803 5.920850720841802 4.920850720841802
6.121092585280919 6.121092585280919 5.121092585280919
6.16776009204485 6.16776009204485 5.16776009204485


In [39]:
# Add in measured frequencies if we know them
# D5
f_meas = (5.80, 5.89, 6.01, 6.14)
f_q_meas = (4.58, 5.31, 4.17, 4.77)
# E5
#f_meas = (5.81, 5.91, 6.03, 6.13)
#f_q_meas = (4.76, 6.26, 4.75, 4.83)
for i,q in enumerate(selectedQubits):
    q.omega_r = 2*pi*1e9*f_meas[i]
    q.omega_q = 2*pi*1e9*f_q_meas[i]

## Qubit parameters ##
From Ted Thorbeck's notes:  
$E_c = \frac{e^2}{2C}$, $E_c/\hbar=\alpha=\text{anharmonicity}$  
$E_J = \frac{I_o \Phi_0}{2 \pi} $  
$\omega_q = \sqrt{8E_JE_c}/\hbar $  
$g = \frac{1}{2} \frac{C_g}{\sqrt{(C_q+C_g)(C_r+C_g)}}\sqrt{\omega_r\omega_q}$  
  
We want g in the range 25-200MHz for an ideal anharmonicity $\alpha$=250MHz  

In [7]:
qb = deepcopy(qubits[1])
g = 2*pi*30e6     # qubit-resonator coupling in Hz

In [41]:
print('Range of C_q on the mask:')
print "C_q = 30fF: E_c = {:.2f}MHz".format( qb.E_c(30e6)/(2*pi*hbar)*1e15 )
print "C_q = 95fF: E_c = {:.2f}MHz".format( qb.E_c(95e6)/(2*pi*hbar)*1e15 )
print
print('Ideal:')
print "Transmon:          E_c = 250MHz: C_sigma = C_q + C_g = {:.2f}fF".format( e**2/2/250e6/(2*pi*hbar)*1e15 )
print "Charge Sensitive:  E_c = 385MHz: C_sigma = C_q + C_g = {:.2f}fF".format( e**2/2/410e6/(2*pi*hbar)*1e15 )

Range of C_q on the mask:
C_q = 30fF: E_c = 645.67MHz
C_q = 95fF: E_c = 203.90MHz

Ideal:
Transmon:          E_c = 250MHz: C_sigma = C_q + C_g = 77.48fF
Charge Sensitive:  E_c = 385MHz: C_sigma = C_q + C_g = 47.24fF


In [123]:
# What variation in C_g should be included on mask for the C_q variation we have?
print( '{:>7}  {:>9}  {:>7}'.format('C_q', 'g', 'C_g') )
for C_q_ in [85e-15, 29e-15, e**2/2/250e6]:
    for g_ in [2*pi*25e6, 2*pi*50e6, 2*pi*200e6]:
        qb.C_q = C_q_
        print "{:>5.2f}fF  {:>6.2f}MHz  {:>5.2f}fF".format(
            1e15*C_q_, g_/2/pi/1e6, 1e15*qb.cap_g(g_))

    C_q          g      C_g
85.00fF   25.00MHz   1.65fF
85.00fF   50.00MHz   3.34fF
85.00fF  200.00MHz  14.34fF
29.00fF   25.00MHz   0.97fF
29.00fF   50.00MHz   1.98fF
29.00fF  200.00MHz   8.80fF
 0.00fF   25.00MHz   0.03fF
 0.00fF   50.00MHz   0.13fF
 0.00fF  200.00MHz   2.02fF


In [16]:
# With caps chosen from the mask:
print "{:>11}   {:>10}  {:>10}  {:>10}  {:>10}  {:>10}  {:>10}".format(
    '', 'C_q', 'E_c', 'E_j', 'alpha', 'g', 'C_g')
for q in qubits:
    print "{:>11}:  {:8.2f}fF  {:7.2f}MHz  {:7.2f}GHz  {:7.2f}MHz  {:7.2f}MHz  {:8.2f}fF".format(
        q.name, 1e15*q.C_q, -q.E_c()/(2*pi*hbar)/1e6, q.E_j()/2/pi/hbar/1e9, q.alpha(q.E_c(),q.E_j())/(2*pi)/1e6, g/2/pi/1e6, 1e15*q.cap_g(g))

                     C_q         E_c         E_j       alpha           g         C_g
OCSQubit040:     46.30fF  -405.23MHz     9.13GHz  -552.59MHz    30.00MHz      1.59fF
OCSQubit050:     47.00fF  -399.88MHz     9.24GHz  -539.17MHz    30.00MHz      1.57fF
OCSQubit060:     46.90fF  -400.46MHz     9.22GHz  -540.60MHz    30.00MHz      1.57fF
OCSQubit070:     47.00fF  -399.80MHz     9.24GHz  -538.97MHz    30.00MHz      1.57fF
OCSQubit080:     47.00fF  -399.72MHz     9.24GHz  -538.77MHz    30.00MHz      1.57fF
OCSQubit090:     47.00fF  -399.47MHz     9.24GHz  -538.16MHz    30.00MHz      1.58fF
OCSQubit100:     46.90fF  -400.87MHz     9.22GHz  -541.62MHz    30.00MHz      1.60fF
OCSQubit108:     46.90fF  -400.38MHz     9.23GHz  -540.40MHz    30.00MHz      1.62fF


In [17]:
# We choose the closest g capacitance from the mask
print "{:>11}   {:>10}  {:>10}  {:>10}  {:>10}  {:>7}  {:>10}  {:>9}  {:>10}  {:>9}".format(
    '', 'C_g', 'g', 'Chi_0/2pi', 'Chi/2pi', 'Q_r', 'kappa', '1/kappa', 'I_c', 'n_crit')
for q in selectedQubits:
    print "{:>11}:  {:>8.2f}fF  {:>7.2f}MHz  {:>7.2f}MHz  {:>7.2f}MHz  {:>7.0f}  {:>7.2f}MHz  {:>7.0f}ns  {:>8.2f}nA  {:>9.0f}".format(
        q.name, 1e15*q.cap_g(q.g()), q.g()/2/pi/1e6, 1e-6*q.Chi_0()/2/pi, 1e-6*q.Chi()/2/pi, q.Q_r(), 
        q.omega_r/q.Q_r()*1e-6/2/pi, q.Q_r()/q.omega_r*1e9, q.I_c()*1e9, ((q.omega_q-q.omega_r)/2/q.g())**2)

                     C_g           g   Chi_0/2pi     Chi/2pi      Q_r       kappa    1/kappa         I_c     n_crit
OCSQubit090:      1.49fF    27.03MHz     0.73MHz    -0.26MHz    11281     0.52MHz      306ns     17.66nA        342
OCSQubit050:      1.44fF    26.94MHz     0.73MHz    -0.26MHz    11540     0.51MHz      310ns     18.06nA        344
OCSQubit108:      1.48fF    27.83MHz     0.77MHz    -0.27MHz    11403     0.54MHz      296ns     19.41nA        323
OCSQubit070:      1.45fF    28.85MHz     0.83MHz    -0.29MHz    10758     0.57MHz      278ns     19.76nA        300


In [18]:
delta = 380e-6; #2\Delta/e in V
Jc = 1e8*673e-9 # A/cm^2
nJJs = (2,1,1,2)
print( '{:>8}   {:>7}  {:>6}   {:>13}'.format('', 'I_c', 'R_N', 'width') )
for i,q in enumerate(selectedQubits):
    print("{}:  {:>5.2f}nA  {:>5.2f}k   {} x {:.3f}nm".format(
        q.name, q.I_c()*1e9, 1e-3*pi/4*delta/q.I_c(), nJJs[i], 1e9*q.I_c()/(1e4*Jc)/100e-9/nJJs[i] ))

               I_c     R_N           width
OCSQubit090:  17.66nA  16.90k   2 x 131.228nm
OCSQubit050:  18.06nA  16.53k   1 x 268.359nm
OCSQubit108:  19.41nA  15.38k   1 x 288.380nm
OCSQubit070:  19.76nA  15.11k   2 x 146.785nm


In [19]:
print( '{:>11}    {:>6}  {:>17}'.format('', 'Ej/Ec', 'Charge dispersion') )
for q in selectedQubits:
    print "{}:   {:>6.3f}  {:>15.3f}MHz".format(q.name, q.E_j()/q.E_c(), q.charge_dispersion()/2/pi/hbar/1e6)

                Ej/Ec  Charge dispersion
OCSQubit090:   21.962            4.335MHz
OCSQubit050:   22.433            3.882MHz
OCSQubit108:   24.076            2.652MHz
OCSQubit070:   24.545            2.378MHz


## Feedline with and without crossovers ##

In [27]:
cpw.setKineticInductanceCorrection(False)
print cpw
cpwx = cpwtools.CPWWithBridges(material='al', w=1e6*cpw.w, s=1e6*cpw.s, bridgeSpacing = 250, bridgeWidth = 3, t_oxide=0.16)
cpwx.setKineticInductanceCorrection(False)
print cpwx

Cl = 0.000164468902665 pF/um
Ll = 0.000432928941855 nH/um
vph = 1.18508579536e8 m/s
z0 = 51.3057939391 Ohms
Cl = 0.000190367402031 pF/um
Ll = 0.000432928941855 nH/um
vph = 1.10152763842e8 m/s
z0 = 47.6883194923 Ohms


## Inductive Coupling ##
From [1], we have the dephasing of a qubit:  
$\Gamma_\phi = \eta\frac{4\chi^2}{\kappa}\bar{n}$, where $\eta=\frac{\kappa^2}{\kappa^2+4\chi^2}$, $\bar{n}=\left(\frac{\Delta}{2g}\right)^2$  
$\Gamma_\phi = \frac{4\chi^2\kappa}{\kappa^2+4\chi^2}\left(\frac{\Delta}{2g}\right)^2$  
To maximize the efficiency of readout, we want to maximize the rate of information leaving the system (into the readout chain), or equivilently, maximize dephasing.  
$\partial_\kappa\Gamma_\phi = 0 = -\frac{4\chi^2(\kappa^2-4\chi^2)}{(\kappa^2+4\chi^2)^2}$ when $2\chi=\kappa$.  
$2\chi = \kappa_r = \omega_r/Q_r$  
$ Q_{r,c} = \frac{8Z_0^2}{\pi(\omega M)^2}$  [2]  
We want a $Q_c$ of 3k-30k  
  
[1] Yan et al. The flux qubit revisited to enhance coherence and reproducibility. Nature Communications, 7, 1–9. http://doi.org/10.1038/ncomms12964  
[2] Matt Beck's Thesis, p.39

In [90]:
d = 5
MperL = inductiveCoupling.inductiveCoupling.CalcMutual(cpw.w*1e6, cpw.w*1e6, cpw.s*1e6, cpw.s*1e6, d, 10*cpw.w*1e6)[0]

The inductance per unit length is 5.24e-08 H/m


In [20]:
MperL = 9e-8 # experimentally measured

In [21]:
print( '{:>11}   {:>7}   {:>15}'.format('', 'M', 'coupling length') )
for q in selectedQubits:
    M = 1/(np.sqrt(q.Q_r()*pi/8/cpw.z0()**2)*q.omega_r)
    print "{:>11}:  {:>5.2f}pH   {:>13.2f}um".format(q.name, M*1e12, (M-1e-12)/MperL*1e6)

                    M   coupling length
OCSQubit090:  18.93pH          199.27um
OCSQubit050:  19.39pH          204.31um
OCSQubit108:  17.31pH          181.27um
OCSQubit070:  19.32pH          203.55um


In [107]:
print( '{:>5}  {:>8}'.format('Q_c', 'l_c') )
for q in [3000,6000,9000,15000,21000,27000,33000,99000]:
    print "{:>5}  {:>6.2f}um".format(q,1/(np.sqrt(q*pi/8/cpw.z0()**2)*qubits[2].omega_r)/MperL*1e6)

  Q_c       l_c
 3000  753.92um
 6000  533.10um
 9000  435.27um
15000  337.16um
21000  284.95um
27000  251.31um
33000  227.31um
99000  131.24um


## Purcell Filter ##
Do we even need a purcell filter? [3]  
Without purcell filter: $\kappa_r T_1 \le \left(\frac{\Delta}{g}\right)^2$  
With purcell filter: $\kappa_r T_1 \le \left(\frac{\Delta}{g}\right)^2 \left(\frac{\omega_r}{\omega_q}\right) \left(\frac{2\Delta}{\omega_r/Q_{pf}}\right)^2$  
$\kappa_r = \omega_r/Q_r$  

With the readout resonators spaced ~30MHz appart, we need a bandwidth of at least 4*30MHz=120MHz.  
We have a range of readout resonators from 5-6GHz.  
  
[3] Jeffrey et al. Fast accurate state measurement with superconducting qubits. Physical Review Letters, 112(19), 1–5. http://doi.org/10.1103/PhysRevLett.112.190504

In [87]:
l_curve = 2*pi*50/4
tot_length = l_curve*(1+2+2+2+1)*2 + 4*750 + 2569 + 4*450 + 2*106

purcell = cpwtools.HalfLResonator(cpw,tot_length)
purcell.addCapacitiveCoupling('in', 40e-15)
purcell.addCapacitiveCoupling('out', 130e-15)

print( "f_max = {:.3f}GHz  Q_in = {:.2f}  Q_out = {:.2f}".format( 1e-9*purcell.fl(), purcell.Qc('in'), purcell.Qc('out') ) )
purcell.l = (tot_length + 503*4)*1e-6
print( "f_min = {:.3f}GHz  Q_in = {:.2f}  Q_out = {:.2f}".format( 1e-9*purcell.fl(), purcell.Qc('in'), purcell.Qc('out') ) )
print
print('The measured purcell filter (no crossovers) seems to be 150-200MHz below expected.  This has been accounted for below.')
f0 = (qubits[1].omega_r + qubits[2].omega_r)/2/2/pi
purcell.setLengthFromFreq(f0 + 175e6)   # The measured purcell filter (no crossovers) seems to be 150-200MHz below expected.
print "f = {:.2f}GHz  l = {:.3f}um  offset = {:.3f}um  Q_in = {:.2f}  Q_out = {:.2f}".format( 1e-9*purcell.fl(), purcell.l*1e6, (purcell.l*1e6-tot_length)/4, purcell.Qc('in'), purcell.Qc('out') )
print "V_out/V_in =", (purcell.Qc('in')/purcell.Qc('out'))**0.5
print "{:.2f}% power lost through input".format( 100*purcell.Ql()/purcell.Qc('in') )
print "{:.2f}% power lost through output".format( 100*purcell.Ql()/purcell.Qc('out') )
print "{:.2f}% power lost internally".format( 100*purcell.Ql()/purcell.Qint() )
print
print "The purcell filter frequency goes up by 310MHz when crossovers are added:"
purcellx = deepcopy(purcell)
purcellx.cpw = cpwx
print "f = {:.2f}GHz  l = {:.3f}um  Q_in = {:.2f}  Q_out = {:.2f}".format( 1e-9*purcellx.fl(), purcellx.l*1e6, purcellx.Qc('in'), purcellx.Qc('out') )

f_max = 6.070GHz  Q_in = 222.95  Q_out = 23.70
f_min = 5.022GHz  Q_in = 333.62  Q_out = 34.46

The measured purcell filter (no crossovers) seems to be 150-200MHz below expected.  This has been accounted for below.
f = 6.24GHz  l = 8572.002um  offset = -66.409um  Q_in = 210.01  Q_out = 22.43
V_out/V_in = 3.059694068527067
9.93% power lost through input
92.95% power lost through output
0.00% power lost internally

The purcell filter frequency goes up by 310MHz when crossovers are added:
f = 5.87GHz  l = 8572.002um  Q_in = 235.91  Q_out = 27.12


In [88]:
print "Purcell Filter FWHM = {:.2f}MHz".format(2*pi*f0/purcell.Ql()/2/pi/1e6)
print "Purcell Filter Q_l = {:.2f}".format(purcell.Ql())
print
print('T1 Limits:')
print('{:>11}   {:>10}  {:>11}'.format('', 'no purcell', 'yes purcell'))
for q in qubits:
    kappa_r = q.omega_r/q.Q_r()
    Delta = q.omega_q - q.omega_r
    #print "{}: T1 limit (no purcell) = {:.2f}us  T1 limit (purcell) = {:.2f}us".format(
    print "{:>11}:  {:>8.2f}us  {:>9.2f}us".format(
        q.name, (Delta/q.g())**2/kappa_r * 1e6, (Delta/q.g())**2 * (q.omega_r/q.omega_q) * (2*Delta/q.omega_r*purcell.Ql())**2/kappa_r * 1e6 )

Purcell Filter FWHM = 290.98MHz
Purcell Filter Q_l = 20.85

T1 Limits:
              no purcell  yes purcell
OCSQubit108:    548.47us   32906.85us
OCSQubit108:   1279.07us  110664.31us
OCSQubit108:   6196.07us  1150702.29us
OCSQubit108:  14058.96us  3710715.55us


## Loss from XY line ##
From Thorbeck's notes, we have
$R_p = R_s(1+Q_s^2)$ and $C_p = C_s\left(\frac{Q_s^2}{1+Q_s^2}\right)$
where $Q_s = \frac{1}{\omega R_s C_s}$
and the "s" and "p" subscript refer to just the coupling capacitor and Z0 of the line in a series or parallel configuration.  Combining this with the normal LC of the qubit, we can find the loss

In [32]:
C_q = qubits[2].C_q
L_q = 1/(qubits[2].omega_q**2 * C_q)
R_s = 50
C_s = 0.1e-15
Q_s = 1/(qubits[2].omega_q * R_s * C_s)
R_p = R_s*(1 + Q_s**2)
C_p = C_s * Q_s**2/(1 + Q_s**2)
omega = 1/np.sqrt((C_q+C_p)*L_q)
Q_xy = omega*R_p*(C_q+C_p)
print("f: {:.3f}GHz --> {:.3f}GHz".format( 1e-9/np.sqrt(C_q*L_q)/2/pi, 1e-9*omega/2/pi))
print("Q = {:.2f}".format(Q_xy))
print("1/kappa = {:.2f}us".format(1e6*Q_xy/omega))

f: 5.121GHz --> 5.116GHz
Q = 2918252.47
1/kappa = 90.79us
